In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
columns_short = np.array([
    "lp", "v", "GTT", "GTn", "GGn", "Ts", "Tp",
    "T48", "T1", "T2", "P48", "P1", "P2", "Pexh",
    "TIC", "mf", "GT_comp_decay", "GT_turb_decay"
])
Sensor_data_csv = pd.read_csv('data.txt', delimiter ='   ',header = None, names = columns_short)
Sensor_data_csv['time'] = range(len(Sensor_data_csv), 0, -1)
Sensor_data_csv.to_csv("Sensor_data.csv")

In [ ]:
print(Sensor_data_csv.shape)
print(Sensor_data_csv.info())
print(Sensor_data_csv.describe())

In [ ]:

# from scipy.stats import zscore
# z_scores = np.abs(zscore(df.select_dtypes(include=[np.number])))
# df = df[(z_scores < 3).all(axis=1)]

# # 5. Normalize / scale features (important since features have very different units)
# from sklearn.preprocessing import StandardScaler, MinMaxScaler

# scaler = StandardScaler()  # z-score scaling
# df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

# df_scaled.to_csv("cleaned_data.csv", index=False)


Derived Features

In [ ]:
Efficiency_ratios = pd.DataFrame(
    {
        'Fuel to Power': Sensor_data_csv['mf'] / Sensor_data_csv['GTT'],
        'Power to Speed': Sensor_data_csv['GTT'] / Sensor_data_csv['v'],
        'Temperature-Pressure': (Sensor_data_csv['T2'] - Sensor_data_csv['T1']) / (Sensor_data_csv['P2'] - Sensor_data_csv['P1']),
    }
)
Efficiency_ratios['time'] = range(len(Efficiency_ratios), 0, -1)
Efficiency_ratios.to_csv("Efficiency_ratios.csv")

Propeller_Performance = pd.DataFrame(
    {
        'Propeller Imbalance': Sensor_data_csv['Ts'] - Sensor_data_csv['Tp'],
        'Total Propeller Torque': Sensor_data_csv['Ts'] + Sensor_data_csv['Tp'],
        'Torque Ratio': Sensor_data_csv['Ts'] / Sensor_data_csv['Tp']
    }
)
Propeller_Performance['time'] = range(len(Propeller_Performance), 0, -1)
Propeller_Performance.to_csv("Propeller_Performance.csv")


Temperature_Pressure_indicator = pd.DataFrame(
    {
        'CTR': Sensor_data_csv['T2'] / Sensor_data_csv['T1'],
        'TTR': Sensor_data_csv['T48'] / Sensor_data_csv['T2'],
        'PR': Sensor_data_csv['P2'] / Sensor_data_csv['P1'],
        'Expansion Ratio': Sensor_data_csv['P48'] / Sensor_data_csv['P2']
    }
)
Temperature_Pressure_indicator['time'] = range(len(Temperature_Pressure_indicator), 0, -1)
Temperature_Pressure_indicator.to_csv("Temperature_Pressure_indicator.csv")


Engine_Stats = pd.DataFrame(
    {
        'Shaft Torque': Sensor_data_csv['GTT'] / Sensor_data_csv['GTn'],
        'GT speed Ratio': Sensor_data_csv['GGn'] / Sensor_data_csv['GTn'],
        'Fuel per Revolution': Sensor_data_csv['mf'] / Sensor_data_csv['GTn']
    }
)
Engine_Stats['time'] = range(len(Engine_Stats), 0, -1)
Engine_Stats.to_csv("Engine_Stats.csv")


Ship_features = pd.DataFrame(
    {
        'Power per Knot': Sensor_data_csv['GTT'] * Sensor_data_csv['GTn'] / Sensor_data_csv['v'],
        'SFC': Sensor_data_csv['mf'] / Sensor_data_csv['v']
    }
)
Ship_features['time'] = range(len(Ship_features), 0, -1)
Ship_features.to_csv("Ship_features.csv")

KPI = pd.DataFrame(
    {
        'fuel_per_knot': Sensor_data_csv['mf'] / Sensor_data_csv['v'],
        'fuel_per_torque': Sensor_data_csv['mf'] / Sensor_data_csv['GTT'],
        'fuel_per_revolution': Sensor_data_csv['mf'] / Sensor_data_csv['GTn'],
        'propeller_imbalance': Sensor_data_csv['Ts'] - Sensor_data_csv['Tp'],
        'compressor_temp_ratio': Sensor_data_csv['T2'] / Sensor_data_csv['T1'],
        'turbine_temp_ratio': Sensor_data_csv['T48'] / Sensor_data_csv['T2'],
        'compressor_pressure_ratio': Sensor_data_csv['P2'] / Sensor_data_csv['P1'],
        'expansion_ratio': Sensor_data_csv['P48'] / Sensor_data_csv['P2'],
        'torque_per_rpm': Sensor_data_csv['GTT'] / Sensor_data_csv['GTn'],
        'power_per_knot': Sensor_data_csv['GTT'] * Sensor_data_csv['GTn'] / Sensor_data_csv['v'],
        'tic_efficiency': Sensor_data_csv['TIC'] / Sensor_data_csv['mf']
    }
)
KPI['time'] = range(len(KPI), 0, -1)
KPI.to_csv("KPI.csv")



Analysis of fuel and efficiency

In [ ]:
#sns.lineplot(data=Sensor_data_csv, x='time', y='mf')

Sensor_data_csv['mf_rolling'] = Sensor_data_csv['mf'].rolling(window=50).mean()
sns.lineplot(data=Sensor_data_csv, x=Sensor_data_csv.index, y='mf_rolling')

In [ ]:
sns.lineplot(data =Sensor_data_csv ,x='v', y='mf')

In [ ]:

Sensor_data_csv['fuel_per_torque'] = Sensor_data_csv['mf'] / Sensor_data_csv['GTT']      # Fuel flow per unit torque
Sensor_data_csv['fuel_per_knot']   = Sensor_data_csv['mf'] / Sensor_data_csv['v']
efficiency = Sensor_data_csv.groupby('v').agg({
    'fuel_per_knot': 'mean',
    'fuel_per_torque': 'mean'
}).reset_index()

sns.lineplot(x='v',y='fuel_per_knot',data = efficiency, marker='o', label='Fuel per Knot')


In [ ]:
sns.lineplot(x='v',y='fuel_per_torque',data = efficiency, marker='x', label='Fuel per Torque')

In [ ]:
# print(Sensor_data_csv['T1'])
# sensor=Sensor_data_csv.drop(['T1',"P1"],axis=1,inplace=False)
# correlation=sensor.corr()
# plt.figure(figsize=(15,15))
# sns.heatmap(correlation,cmap="coolwarm",fmt=".2f",annot=True,linewidth=0.5)


In [ ]:
# print(Sensor_data_csv.columns)
# sen2 = Sensor_data_csv.drop(["mf_rolling","fuel_per_knot","fuel_per_torque"], axis =1, inplace = False)
# for col in sen2.columns:
#     plt.figure(figsize=(8,6))
#     Sensor_data_csv.boxplot(column=[col])
#     plt.title(f"boxplot of {col}")
#     plt.show()

# Sensor_data_csv = Sensor_data_csv.drop(["mf_rolling","fuel_per_knot","fuel_per_torque"], axis =1, inplace = True)

In [ ]:
alerts = []

# 1. Fuel Efficiency
alerts.append(
    KPI[KPI['fuel_per_knot'] > 0.3]
    .assign(alert_type="Fuel_Efficiency", severity="WARNING")
)

# 2. Fuel per Torque
alerts.append(
    KPI[KPI['fuel_per_torque'] > 0.05]
    .assign(alert_type="Fuel_Torque_Inefficiency", severity="WARNING")
)

# 3. Fuel per Revolution
alerts.append(
    KPI[KPI['fuel_per_revolution'] > 0.001]
    .assign(alert_type="Fuel_Revolution_Inefficiency", severity="INFO")
)

# 4. Propeller Imbalance (> 0)
alerts.append(
    KPI[KPI['propeller_imbalance'] != 0]
    .assign(alert_type="Torque_Imbalance", severity="CRITICAL")
)

# 5. Compressor Pressure Ratio (outside 10–15)
alerts.append(
    KPI[(KPI['compressor_pressure_ratio'] < 10) | (KPI['compressor_pressure_ratio'] > 15)]
    .assign(alert_type="Compressor_Pressure_Anomaly", severity="WARNING")
)

# 6. Compressor Temp Ratio (outside 1.1–2.5)
alerts.append(
    KPI[(KPI['compressor_temp_ratio'] < 1.1) | (KPI['compressor_temp_ratio'] > 2.5)]
    .assign(alert_type="Compressor_Temp_Anomaly", severity="WARNING")
)

# 7. Turbine Temp Ratio (> 6 = overheating)
alerts.append(
    KPI[KPI['turbine_temp_ratio'] > 6]
    .assign(alert_type="Turbine_Overheating", severity="CRITICAL")
)

# 8. Expansion Ratio (< 1 = anomaly)
alerts.append(
    KPI[KPI['expansion_ratio'] < 1]
    .assign(alert_type="Expansion_Anomaly", severity="WARNING")
)

# 9. Torque per RPM (> 5 = stress)
alerts.append(
    KPI[KPI['torque_per_rpm'] > 5]
    .assign(alert_type="Torque_Stress", severity="CRITICAL")
)

# 10. Power per Knot (> 5000 = inefficient)
alerts.append(
    KPI[KPI['power_per_knot'] > 5000]
    .assign(alert_type="Power_Inefficiency", severity="WARNING")
)

# 11. TIC Efficiency (< 0.1 or > 10 = anomaly)
alerts.append(
    KPI[(KPI['tic_efficiency'] < 0.1) | (KPI['tic_efficiency'] > 10)]
    .assign(alert_type="TIC_Anomaly", severity="WARNING")
)

# Combine all alerts into one DataFrame
alerts_df = pd.concat(alerts, ignore_index=True)

# Show summary
print(alerts_df[['alert_type', 'severity']].value_counts())


In [ ]:
Sensor_data_csv.columns

In [ ]:
KPI.info()

In [ ]:
import requests
import json

API_URL = "http://127.0.0.1:8000/add_bulk"

Sensor_data_csv = Sensor_data_csv.rename(columns={
    "GT_comp_decay": "decay_comp",
    "GT_turb_decay": "decay_turbine"
})
print(Sensor_data_csv.info())
required_cols = [
    "lp","v","GTT","GTn","GGn","Ts","Tp","T48","T1","T2",
    "P48","P1","P2","Pexh","TIC","mf","decay_comp","decay_turbine","time"
]

Sensor_data_csv = Sensor_data_csv[required_cols]

API_data = Sensor_data_csv.to_dict(orient="records")

response = requests.post(API_URL, json=API_data)

print(response)




In [ ]:
import pandas as pd
import requests

KPI_data_csv = pd.read_csv("KPI.csv")

print(KPI_data_csv.info())

required_cols = [
    "fuel_per_knot",
    "fuel_per_torque",
    "fuel_per_revolution",
    "propeller_imbalance",
    "compressor_temp_ratio",
    "turbine_temp_ratio",
    "compressor_pressure_ratio",
    "expansion_ratio",
    "torque_per_rpm",
    "power_per_knot",
    "tic_efficiency",
]

KPI_data_csv = KPI_data_csv[required_cols]

API_data = KPI_data_csv.to_dict(orient="records")

API_URL = "http://127.0.0.1:8000/add_KPI"

response = requests.post(API_URL, json=API_data)

print(response)
print(response.text)
